In [1]:
# Passo 1: Importar as bibliotecas necessárias

import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Passo 2: Carregar os dados da planilha 

# Definir o caminho para o arquivo CSV
caminho_arquivo = 'C:/TCC/pedidos_faturamento.csv'

# Carregar o arquivo CSV em um DataFrame
df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='utf-8', header=0)

# Exibir as primeiras linhas do DataFrame para verificar se foi carregado corretamente
print(df.head())

   Código do Item Cidade de Entrega Estado de Entrega  \
0               1           FORMIGA                MG   
1               1           FORMIGA                MG   
2               3           FORMIGA                MG   
3               4           FORMIGA                MG   
4               5           FORMIGA                MG   

  Data de Implantação do Pedido Data de Entrega do Pedido  Tempo de Entrega  
0                    20/05/2020                04/06/2020                15  
1                    22/05/2020                25/05/2020                 3  
2                    22/05/2020                25/05/2020                 3  
3                    18/06/2020                06/07/2020                18  
4                    18/06/2020                06/07/2020                18  


In [3]:
# passo 3: Definir a coluna alvo
target_column = 'Tempo de Entrega'

# Separar as features e o target
X = df.drop(columns=[target_column])
y = df[target_column]

# Verificar os tipos de dados
print(X.dtypes)

# Identificar colunas categóricas e numéricas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[float, int]).columns.tolist()

# Imprimir colunas categóricas e numéricas para verificação
print(f"Colunas categóricas: {categorical_cols}")
print(f"Colunas numéricas: {numerical_cols}")

Código do Item                    int64
Cidade de Entrega                object
Estado de Entrega                object
Data de Implantação do Pedido    object
Data de Entrega do Pedido        object
dtype: object
Colunas categóricas: ['Cidade de Entrega', 'Estado de Entrega', 'Data de Implantação do Pedido', 'Data de Entrega do Pedido']
Colunas numéricas: ['Código do Item']


In [20]:
# passo 4: Criar transformações para colunas numéricas e categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Criar pipeline que inclui preprocessor e o modelo AdaBoost
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', AdaBoostRegressor(n_estimators=100, learning_rate=2.0, random_state=42))
])


In [21]:
# passo 5: Configurar a validação cruzada com k-fold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [22]:
# passo 6: Avaliar o modelo com cross-validation
mse_scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_mean_squared_error')
mae_scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_mean_absolute_error')
r2_scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='r2')

In [25]:
# passo 7: Converter scores para valores positivos
mse_scores = -mse_scores
mae_scores = -mae_scores

print("AdaBoost - Cross-Validation Results")
print(f"Mean Squared Error: {mse_scores.mean():.2f} ± {mse_scores.std():.2f}")
print(f"Mean Absolute Error: {mae_scores.mean():.2f} ± {mae_scores.std():.2f}")
print(f"R2 Score: {r2_scores.mean():.2f} ± {r2_scores.std():.2f}")

AdaBoost - Cross-Validation Results
Mean Squared Error: 2507.59 ± 1450.71
Mean Absolute Error: 44.21 ± 15.61
R2 Score: -4.10 ± 2.98
